<a href="https://colab.research.google.com/github/hajihye123/Machine_Learning_Study/blob/main/%ED%95%98%EC%A7%80%ED%98%9C_HW_11_1_Spam_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 11-1. Simple Naive Bayes Classifier


## T1. Load a dataset

The following code loads a dataset consisting of text messages and spam-ham labels.

You can write your own code below the **"TODOs"** to answer the questions.



### Questions:
* Number of spam messges? [*747*]
* Number of ham messages? [*4825*]

In [ ]:
from typing import List, Tuple, Dict, Iterable, Set
from collections import defaultdict
import re
import math
import pandas as pd

url = 'https://raw.githubusercontent.com/mlee-pnu/IDS/main/spam_dataset.csv'
df = pd.read_csv(url)

# TODOs
print(df.head())

message = df.groupby(['Category']).count() 
print(message)

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
          Message
Category         
ham          4825
spam          747


## T2. Spam filter for individual words

We first defined a function ***tokenize()*** to convert a given text into a set of words. 

Using the function, we now try to count the frequency of each word in each class (spam and ham).

Complete the following code and answer the following questions:
 



### Qeustions: 
*   Number of spam messages containing the word "free": [*170*]

Let's assume we only care for the word "free" to determine if a message is a spam or not. Answer the following questions:

*   P ( *ham* | *free* ) = [*0.2576419213973799*]
*   Is this message spam? [*Yes*]

*Note: Do not apply a smoothing method here.*



In [ ]:
def tokenize(text: str) -> Set[str]:
    text = text.lower()                         
    all_words = re.findall("[a-z0-9']+", text)  
    return set(all_words)                       

In [ ]:
tokens: Set[str] = set()
token_spam_counts: Dict[str, int] = defaultdict(int)
token_ham_counts: Dict[str, int] = defaultdict(int)

spam = df[df.Category == 'spam']
ham = df[df.Category == 'ham']

for msg in spam['Message'].to_list():
  for token in tokenize(msg):
    tokens.add(token)
    token_spam_counts[token] += 1

for msg in ham['Message'].to_list():
  for token in tokenize(msg):
    tokens.add(token)
    token_ham_counts[token] += 1

# TODOs
word = "free"
n_word_spam = token_spam_counts[word] # frequency of the word in spam messages
n_word_ham = token_ham_counts[word] # frequency of the word in ham messages

p_spam = spam['Message'].count() / (spam['Message'].count() + ham['Message'].count()) # P(spam)
p_ham =  ham['Message'].count() / (spam['Message'].count() + ham['Message'].count()) # P(ham)

p_word_given_spam = n_word_spam / spam['Message'].count() # P(word|spam)
p_word_given_ham = n_word_ham / ham['Message'].count() # P(word|ham)

# P(spam|word)
p_spam_given_word = p_word_given_spam * p_spam / ( (n_word_spam + n_word_ham) / (spam['Message'].count() + ham['Message'].count()))
print(p_spam_given_word)
# P(ham|word)
p_ham_given_word = p_word_given_ham * p_ham / ( (n_word_spam + n_word_ham) / (spam['Message'].count() + ham['Message'].count()))
print(p_ham_given_word)

0.74235807860262
0.2576419213973799


## T3. Spam filter that combines words: Naive Bayes

You received a text message "just do it" from an unknown sender.

Complete the function ***predict()*** that outputs the probability of the message being spam and the predicted label of the message. 


### Questions:

*   P ( *spam* | *text* ) = [*0.002791028835884658*]
*   Is this text message spam? [*No*]

*Note: You do not need to apply a smoothing method here.*



In [ ]:
text = "just do it"

# TODOs
def predict(text: str):
  message = text.split()
  for word in message:
    n_word_spam = token_spam_counts[word]
    n_word_ham = token_ham_counts[word]
    p_spam = spam['Message'].count() / (spam['Message'].count() + ham['Message'].count()) # P(spam)
    p_ham =  ham['Message'].count() / (spam['Message'].count() + ham['Message'].count()) # P(ham)
    p_word_given_spam = n_word_spam / spam['Message'].count() # P(word|spam)
    p_word_given_ham = n_word_ham / ham['Message'].count() # P(work|ham)
    p_spam_given_word = p_word_given_spam * p_spam / ( (n_word_spam + n_word_ham) / (spam['Message'].count() + ham['Message'].count())) # P(spam|word)
    p_ham_given_word = p_word_given_ham * p_ham / ( (n_word_spam + n_word_ham) / (spam['Message'].count() + ham['Message'].count())) # P(ham|word)
    p_spam_given_word *= p_spam_given_word
    p_ham_given_word *= p_ham_given_word
  if p_ham_given_word > p_spam_given_word:
    print('P(spam|text):',p_spam_given_word) # P(spam|text)
    label = "ham"
    return p_ham_given_word, label
  elif p_ham_given_word < p_spam_given_word:
    label = "spam"
    return p_spam_given_word, label
  else:
    label = "equal"
    return p_ham_given_word, label

print(predict(text))

P(spam|text): 0.002791028835884658
(0.8971306514773941, 'ham')


## T4. Smoothing method

You again received two text messages from unknown senders.

Complete the function ***spamFilter()*** that classifies a given message. 

You may want to apply a smoothing method for this task.


### Questions:

*   Is textA spam?: [*Yes*]
*   Is textB spam?: [*No*]


In [ ]:
textA = "reward! download your free ticket from our website www.pnu.edu"
textB = "call me and get your money back"

# TODOs
n_spam = sum(token_spam_counts.values())
n_ham = sum(token_ham_counts.values())

# n_tokens
n_tokens = len(tokens)

# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in tokens}
parameters_ham = {unique_word:0 for unique_word in tokens}

alpha = 1

# Calculate parameters
for word in tokens:
  n_word_spam = token_spam_counts[word] # frequency of the word in spam messages
  p_word_given_spam = (n_word_spam + alpha) / (n_spam + alpha * n_tokens)
  parameters_spam[word] = p_word_given_spam

  n_word_ham = token_ham_counts[word] # frequency of the word in spam messages
  p_word_given_ham = (n_word_ham + alpha) / (n_ham + alpha * n_tokens)
  parameters_ham[word] = p_word_given_ham

def spamFilter(text: str):
  message = text.split()
  
  p_spam_given_message = p_spam
  p_ham_given_message = p_ham

  for word in message:
    if word in parameters_spam:
      p_spam_given_message *= parameters_spam[word]

    if word in parameters_ham: 
      p_ham_given_message *= parameters_ham[word]

  print('P(spam|message):', p_spam_given_message)
  print('P(ham|message):', p_ham_given_message)

  if p_ham_given_message > p_spam_given_message:
    label = "ham"
    return label
  elif p_ham_given_message < p_spam_given_message:
    label = "spam"
    return label
  else:
    label = "equal"
    return label
  
print(spamFilter(textA))
print(spamFilter(textB))

P(spam|message): 9.950760035451472e-23
P(ham|message): 4.566291033531711e-24
spam
P(spam|message): 2.7050714264181724e-20
P(ham|message): 6.1048710578548686e-18
ham
